In [1]:
!pip install pandas -q
!pip install denseweight

import numpy as np
import pandas as pd
from denseweight import DenseWeight


# File paths
train_file = "./extraextra_lin_normalized_log1p_train_data.csv"
test_file = "./extraextra_lin_normalized_log1p_test_data.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

dw = DenseWeight(alpha=1)
print(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy().shape)
weights = dw.fit(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy())
train_weights = dw(train_df["Label"].to_numpy())
test_weights = dw(test_df["Label"].to_numpy())

print(dw([0.01]))
print(dw([0.1]))
print(dw([1]))
print(dw([6]))
print(dw([200]))
print(dw([400]))

# Display the first few rows of each dataset
print("Train Data:")
print(train_df.info())

print("\nTest Data:")
print(test_df.info())



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


(1000000,)
[0.00696068]
[1.77102832]
[2.20667524]
[2.23723705]
[2.23945467]
[2.23945467]
Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  800000 non-null  float64
 1   Series2  800000 non-null  float64
 2   Series3  800000 non-null  float64
 3   Label    800000 non-null  float64
dtypes: float64(4)
memory usage: 24.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  200000 non-null  float64
 1   Series2  200000 non-null  float64
 2   Series3  200000 non-null  float64
 3   Label    200000 non-null  float64
dtypes: float64(4)
memory usage: 6.1 MB
None


In [2]:
train_df.describe()

,Series1,Series2,Series3,Label
count,800000.000000,800000.000000,800000.000000,8.000000e+05
mean,0.499820,0.500057,0.500170,2.498894e-01
std,0.219353,0.291573,0.291520,5.131788e-01
min,0.000000,0.000000,0.000000,0.000000e+00
25%,0.387755,0.252525,0.252525,4.289651e-09
50%,0.499823,0.494949,0.505051,4.391191e-02
75%,0.612245,0.757576,0.757576,3.308511e-01
max,1.000000,1.000000,1.000000,5.991320e+00


In [3]:
# install norse and pytorch
!pip install sinabs


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.inputval = dataframe["Series1"]
        self.mean = dataframe["Series2"]
        self.std = dataframe["Series3"]
        self.labels = dataframe["Label"]

    def __len__(self):
        return len(self.inputval)

    def __getitem__(self, idx):
        inputval = self.inputval.iloc[idx]
        mean = self.mean.iloc[idx]
        std = self.std.iloc[idx]
        labels = self.labels.iloc[idx]
        return inputval, mean, std, labels
    
# Create the datasets
train_dataset = DataFrameDataset(train_df)
test_dataset = DataFrameDataset(test_df)


In [5]:
from torch.utils.data import WeightedRandomSampler

train_sampler = WeightedRandomSampler(weights=train_weights, num_samples=len(train_weights))


In [6]:
# Encode input data
train_loader = DataLoader(train_dataset, batch_size=128, sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=128) #, shuffle=True)


In [7]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
import torch

TRAINING_PREFIX = "denseweight_1_0_extraextra_lin_normalized_256_2HL_normalized_log1p"

ann = nn.Sequential(
    nn.Linear(3, 256),  # Input layer: 3 features (mu, sigma, x)
    nn.ReLU(),
    nn.Linear(256, 256),  # first hidden layer
    nn.ReLU(),
    nn.Linear(256, 256),  # second hidden layer
    nn.ReLU(),
    nn.Linear(256, 1)    # Output layer: single value for f(x; mu, sigma)
)

# Load checkpoint
#ann.load_state_dict(torch.load(f"results/{TRAINING_PREFIX}/ann_epoch_300.pth"))

ann.to(device)

total_params = sum(p.numel() for p in ann.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params}")


Total Parameters: 132865


In [9]:
from sinabs.from_torch import from_model

num_time_steps_per_sample = 100

sinabs_model = from_model(ann, input_shape=(3,), add_spiking_output=True, synops=False, num_timesteps=num_time_steps_per_sample)
sinabs_model

Network(
  (spiking_model): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (spike_output): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
  )
  (analog_model): Sequential(
    (0): Linear(in_features=3, out_features

In [10]:
!pip install torcheval -q

from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from torcheval.metrics import R2Score


def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Eval Loss', line=dict(color='orange')))

    # Configure layout
    fig.update_layout(title='Training and Evaluation Losses',
                    xaxis_title='Epoch',
                    yaxis_title='Loss',
                    template='plotly_dark')

    # Display the figure widget
    display(fig)
    return fig

def update_loss_plot(fig, train_loss, eval_loss, from_epoch=0):
    if from_epoch != 0:
        with fig.batch_update():
            fig.data[0].x = list(range(from_epoch, len(train_loss)))
            fig.data[0].y = train_loss[from_epoch:]
            fig.data[1].x = list(range(from_epoch, len(eval_loss)))
            fig.data[1].y = eval_loss[from_epoch:]
    elif len(train_loss) < 30:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss)))
            fig.data[0].y = train_loss
            fig.data[1].x = list(range(len(eval_loss)))
            fig.data[1].y = eval_loss
    else:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss) - 30, len(train_loss)))
            fig.data[0].y = train_loss[-30:]
            fig.data[1].x = list(range(len(eval_loss) - 30, len(eval_loss)))
            fig.data[1].y = eval_loss[-30:]        
    

import numpy as np
def gaussian_probability(x, y, z):
    return (1 / (np.sqrt(2 * np.pi) * z)) * np.exp(-((x - y) ** 2) / (2 * z ** 2))

from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def do_epoch(loader, model, optimizer, device, n_epochs: int, current_epoch: int, train: bool = False, scheduler=None):
    if train:
        model.train()
    else:
        model.eval()
    epoch_loss = 0
    metric = R2Score(device=device)
    for inputvals, means, stds, labels in tqdm(loader, desc=f"{'Epoch' if train else 'Eval Epoch'} {current_epoch+1}/{n_epochs}"):
        inputs = torch.stack((inputvals, means, stds), dim=1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        metric.update(outputs.squeeze(), labels)
        if train:
            optimizer.zero_grad()
            loss = F.huber_loss(outputs.squeeze(), labels.float())
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
        else:
            loss = F.huber_loss(outputs.squeeze(), labels.float())
        epoch_loss += loss.item()
    
    # For Plateau scheduler
    if scheduler is not None and not train:
        print("sched step")
        scheduler.step(epoch_loss)
    return epoch_loss, metric.compute()


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# Scheduler
!pip install pytorch_warmup

from pytorch_warmup import LinearWarmup

# Train

ann.train()

N_EPOCHS = 1000
LR = 0.0004
SAVE_EVERY = 50

# Create subfolder for this loop
import os
import shutil

if os.path.exists(f"./results/{TRAINING_PREFIX}"):
    shutil.rmtree(f"./results/{TRAINING_PREFIX}")
os.makedirs(f"./results/{TRAINING_PREFIX}")

optim = torch.optim.AdamW(ann.parameters(), lr=LR)
#optim = torch.optim.SGD(ann.parameters(), lr=LR)
#opt_step = torch.compile(optim.step, mode="reduce-overhead")

num_steps = len(train_loader) * N_EPOCHS

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.25, patience=10, verbose=True)
#    optim,
#    milestones=[i * len(train_loader) for i in [250, 500]])
#warmup_scheduler = LinearWarmup(optim, warmup_period=(len(train_loader) * 50) )


last_loss = 0
last_eval_loss = 0

epoch_loss_list = []
eval_loss_list = [] 

fig = create_loss_plot()

for epoch in range(N_EPOCHS):
    if (epoch + 1) % 30 == 0:
        clear_output(wait=True)
        fig = create_loss_plot()
        update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    
    epoch_loss, train_R2_score = do_epoch(train_loader, ann, optim, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=True, scheduler=lr_scheduler)
    eval_epoch_loss, eval_R2_score = do_epoch(test_loader, ann, None, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=False, scheduler=lr_scheduler)
    eval_loss_list.append(eval_epoch_loss/len(test_loader))
    print(f"Epoch {epoch+1}, LR: {lr_scheduler.get_last_lr()}, Train Loss: {epoch_loss/len(train_loader)}, Diff: {epoch_loss/len(train_loader) - last_loss}, Eval Loss: {eval_epoch_loss/len(test_loader)}, Diff Eval: {eval_epoch_loss/len(test_loader) - last_eval_loss}, Train R2 Score: {train_R2_score}, Eval R2 Score: {eval_R2_score}")
    last_loss = epoch_loss/len(train_loader)
    last_eval_loss = eval_epoch_loss/len(test_loader)
    epoch_loss_list.append(epoch_loss/len(train_loader))
    update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    if epoch % SAVE_EVERY == 0:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth")
    elif epoch == N_EPOCHS - 1:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth") 

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '2fe0a135-ca04-4e56-80a7-0563d72655d0',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': 'c7876bab-ac8e-4135-aa10-ae24b8dbb569',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

Epoch 90/1000:   0%|          | 0/6250 [00:00<?, ?it/s]

Eval Epoch 90/1000:   0%|          | 0/1563 [00:00<?, ?it/s]

sched step
Epoch 90, LR: [6.25e-06], Train Loss: 0.006849382420511756, Diff: -8.88098355429981e-05, Eval Loss: 0.003244612449791698, Diff Eval: -4.477129102013061e-06, Train R2 Score: 0.9601941704750061, Eval R2 Score: 0.9677774906158447


Epoch 91/1000:   0%|          | 0/6250 [00:00<?, ?it/s]

...

sched step
Epoch 94, LR: [6.25e-06], Train Loss: 0.006774140571880817, Diff: -0.00018430552279358366, Eval Loss: 0.0033770128163041445, Diff Eval: -0.0002001789291574841, Train R2 Score: 0.9598334431648254, Eval R2 Score: 0.96646648645401


Epoch 95/1000:   0%|          | 0/6250 [00:00<?, ?it/s]

Eval Epoch 95/1000:   0%|          | 0/1563 [00:00<?, ?it/s]

sched step
Epoch 95, LR: [6.25e-06], Train Loss: 0.006810826996888682, Diff: 3.668642500786502e-05, Eval Loss: 0.0032593916146696415, Diff Eval: -0.00011762120163450298, Train R2 Score: 0.9598965048789978, Eval R2 Score: 0.9674987196922302


Epoch 96/1000:   0%|          | 0/6250 [00:00<?, ?it/s]

In [ ]:
#torch.save(ann.state_dict(), "largerer_ann_new_data5.pth")


In [ ]:
fig = create_loss_plot()
update_loss_plot(fig, epoch_loss_list, eval_loss_list, 10)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '29098738-dc2b-4dde-a3c9-6405766ad9d4',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': '0c64bd97-2f9b-42b0-820c-6b9d14926bf6',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

In [ ]:
epoch_loss_list

[0.02354570327594876,
 0.016079609875362368,
 0.013032140182000584,
 0.011524268243256957,
 0.010171118808425963,
 0.009753358722217382,
 0.00949931381121045,
 0.008910709847644903,
 0.008775201361898799,
 0.008207614116957411,
 0.008266430261835921,
 0.007838183197292965,
 0.008113450737993699,
 0.007735621863713022,
 0.007821641203609762,
 0.007920708534969017,
 0.007775321107667405,
 0.00760134226844646,
 0.007880219091107138,
 0.007694296670369804,
 0.0071474009543657305,
 0.007292596587599255,
 0.006992759147346951,
 0.0067663737699622285,
 0.006963065523775294,
 0.006801811356347753,
 0.00693427745963214,
 0.00679407113855239,
 0.006699170251275645,
 0.006832049342935206,
 0.006571169172923546,
 0.006604561636155704,
 0.006860836541986792,
 0.006776035700095818,
 0.007147561122267507,
 0.004572304146295646,
 0.004483162216356722,
 0.004515452351988061,
 0.004457874274796341,
 0.004530045843842672,
 0.004508944094426697,
 0.004500302514513024,
 0.00448892230636091,
 0.004555785058

In [ ]:
import pickle

with open(f"./extraextra_lin_normalized_log1p_scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [ ]:
ann.eval()
transformed = scaler.transform([[0.5, 0.5, 0.5]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())


c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(0.8993888616979944)

In [ ]:
gaussian_probability(0.5, 0.5, 0.5)

np.float64(0.7978845608028654)

In [ ]:
transformed = scaler.transform([[0.1, 0.1, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(4.639858601185763)

In [ ]:
gaussian_probability(0.1, 0.1, 0.1)

np.float64(3.989422804014327)

In [ ]:
transformed = scaler.transform([[0.01, 0.01, 0.01]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(42.19871407709524)

In [ ]:
gaussian_probability(0.01, 0.01, 0.01)

np.float64(39.894228040143275)

In [ ]:
transformed = scaler.transform([[0.9, 0.9, 0.9]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(0.4602683311757058)

In [ ]:
gaussian_probability(0.9, 0.9, 0.9)

np.float64(0.44326920044603635)

In [ ]:
transformed = scaler.transform([[0.9, 0.7, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(0.5615511663070611)

In [ ]:
gaussian_probability(0.9, 0.7, 0.1)

np.float64(0.53990966513188)

In [ ]:
transformed = scaler.transform([[0.643, 0.7, 0.1]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(3.9600567582951856)

In [ ]:
gaussian_probability(0.643, 0.7, 0.1)

np.float64(3.3912431320419234)

In [ ]:
transformed = scaler.transform([[4, 0.3, 0.2]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(0.0002111121305020598)

In [ ]:
gaussian_probability(4, 0.3, 0.2)

np.float64(9.57716245835995e-75)

In [ ]:
train_dataset.mean.unique()

array([0.56565657, 0.38383838, 0.24242424, 0.71717172, 0.90909091,
       0.25252525, 0.09090909, 0.52525253, 0.57575758, 0.72727273,
       0.67676768, 0.33333333, 0.54545455, 0.85858586, 0.01010101,
       0.84848485, 0.02020202, 0.08080808, 0.6969697 , 0.8989899 ,
       0.63636364, 0.28282828, 0.14141414, 0.03030303, 0.23232323,
       0.87878788, 0.41414141, 0.3030303 , 0.31313131, 0.27272727,
       0.68686869, 0.39393939, 0.5959596 , 0.45454545, 0.19191919,
       0.76767677, 0.93939394, 0.06060606, 0.73737374, 0.07070707,
       0.43434343, 0.16161616, 0.48484848, 0.44444444, 0.86868687,
       0.18181818, 0.49494949, 0.1010101 , 0.82828283, 0.97979798,
       0.13131313, 1.        , 0.2020202 , 0.64646465, 0.74747475,
       0.58585859, 0.62626263, 0.50505051, 0.35353535, 0.51515152,
       0.94949495, 0.77777778, 0.83838384, 0.11111111, 0.34343434,
       0.66666667, 0.75757576, 0.88888889, 0.70707071, 0.53535354,
       0.42424242, 0.78787879, 0.21212121, 0.80808081, 0.81818

In [ ]:
train_dataset.std.unique()

array([0.68686869, 0.23232323, 0.15151515, 0.92929293, 0.57575758,
       0.97979798, 0.        , 0.93939394, 0.04040404, 0.03030303,
       0.98989899, 0.54545455, 0.09090909, 0.42424242, 0.56565657,
       0.34343434, 0.14141414, 0.26262626, 0.01010101, 0.55555556,
       0.71717172, 0.81818182, 0.5959596 , 0.45454545, 0.47474747,
       0.11111111, 0.60606061, 0.61616162, 0.43434343, 0.6969697 ,
       0.75757576, 0.46464646, 0.82828283, 0.32323232, 0.16161616,
       0.91919192, 0.18181818, 0.29292929, 0.63636364, 0.76767677,
       0.96969697, 0.58585859, 0.74747475, 0.86868687, 0.36363636,
       0.90909091, 0.88888889, 0.12121212, 0.27272727, 0.73737374,
       0.62626263, 0.65656566, 0.49494949, 0.87878788, 0.1010101 ,
       0.77777778, 0.8989899 , 0.17171717, 0.2020202 , 0.48484848,
       0.08080808, 0.94949495, 0.24242424, 0.50505051, 0.41414141,
       0.66666667, 0.31313131, 0.39393939, 0.25252525, 0.37373737,
       0.05050505, 0.51515152, 0.21212121, 0.33333333, 1.     

In [ ]:
# std 1.25 and mean -3.04
transformed = scaler.transform([[-2, -3.04, 1.25]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(0.23997641710620973)

In [ ]:
gaussian_probability(-2, -3.04, 1.25)

np.float64(0.22578002723581)

In [ ]:
transformed = scaler.transform([[0.999, 1, 0.001]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(261.7693535578953)

In [ ]:
gaussian_probability(0.999, 1, 0.001)

np.float64(241.97072451914318)

In [ ]:
transformed = scaler.transform([[1, 1, 0.001]])
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

c:\Users\Lutz\Bachelorarbeit2\SNN\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning:

X does not have valid feature names, but MinMaxScaler was fitted with feature names



np.float64(395.1356436863128)

In [ ]:
gaussian_probability(1, 1, 0.001)

np.float64(398.94228040143275)